## List of resources
- https://mysejahtera.malaysia.gov.my/healthfacilities/
- https://www.moh.gov.my/index.php/database_stores/store_view/3
- https://medicalprac.moh.gov.my/v2/modules/mastop_publish/?tac=SENARAI_KLINIK_PERUBATAN_SWASTA
- https://www.vaksincovid.gov.my/json/ppv.json
- https://fh.moh.gov.my/v3/index.php/direktori/direktori-klinik/perak/klinik-kesihatan/

- https://www.codestudyblog.com/cnb11/1125112259.html
- https://towardsdatascience.com/google-maps-feature-extraction-with-selenium-faa2b97b29af
- https://towardsdatascience.com/scraping-google-maps-reviews-in-python-2b153c655fc2
- https://selenium-python.readthedocs.io/index.html
- https://www.bestproxyreviews.com/google-map-scrapers/
- https://medium.com/swlh/scraping-google-maps-using-selenium-3cec08eb6a92
- https://stackoverflow.com/questions/50466238/how-to-bypass-google-recaptcha-for-testing-using-selenium

In [1]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
from geopy.geocoders import Nominatim
import json
import pandas as pd
from collections import OrderedDict
from geopy.geocoders import Nominatim
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import random  
import string
import re
from tqdm import tqdm
import numpy as np

## Test rate limit

In [17]:
class GMap_dump_load(object):
    def __init__(self, old_url):
        self.old_url = old_url

    def __call__(self, driver):
        url = driver.current_url
        if url != old_url:
            return True
        else:
            return False

In [7]:
try:
    driver.close()
except:
    pass

count = 0
driver = webdriver.Firefox(executable_path=r'D:\Program Files\Geckodriver\geckodriver.exe')
driver.get("https://www.google.com.my/maps/search/a")

In [24]:
start = time.time()
while(True):
    try:
        old_url = driver.current_url
        words = ''.join((random.choice(string.ascii_lowercase) for x in range(8)))
        count+=1
        searchbox = driver.find_element_by_xpath('//*[@id="searchboxinput"]')
        searchbox.clear()
        searchbox.send_keys(words)
        searchbox.send_keys(Keys.RETURN)
        # wait = WebDriverWait(driver, 5)
        # element = wait.until(GMap_dump_load(old_url))
        time.sleep(3)  
    except Exception as e:
        print("{0}\nCount: {1}".format(e, count))
        end = time.time()
        print(end - start)
        print(old_url)
        print(driver.current_url)
        break

KeyboardInterrupt: 

## Trial (using concurrent)

In [14]:
files = [
    "A) Senarai Klinik Perubatan Swasta Berdaftar Yang Boleh Menjalankan Saringan COVID-19 Dalam Kawasan Premis",
    "B) Senarai Hospital Swasta Berlesen Yang Boleh Menjalankan Saringan COVID-19 Di Dalam Kawasan Premis",
    "hospital_data",
    "Klinik 1Malaysia",
    "Klinik Desa",
    "Klinik Kesihatan Kerajaan",
    "Klinik Kesihatan Yang Ada Kerjasama Dengan Wakil Komuniti 2017",
    "Lokasi Klinik Kesihatan Swasta di Pulau Pinang",
    "Lokasi Pusat Kesihatan Awam di Negeri Pulau Pinang 2018",
    "Senarai Klinik Ibu Dan Anak Mengikut Negeri Dan Daerah",
    "VAC",
]

In [25]:
pd_list = [pd.read_csv(f"CSV/{file}.csv").replace(np.nan, '', regex=True) for file in files]

In [18]:
try:
    driver.close()
except Exception as e:
    print(e)

Message: Failed to decode response from marionette



In [4]:
# initialize
COOR_PT = re.compile(r"@([\d]+[.][\d]+),([\d]+[.][\d]+),")

driver = webdriver.Firefox()
driver.get("https://www.google.com.my/maps/search/a")

In [5]:
def get_coord(search_text:str):
    latitude = ""
    longitude = ""
    try:
        searchbox = driver.find_element_by_xpath('//*[@id="searchboxinput"]')
        searchbox.clear()
        searchbox.send_keys(search_text)
        searchbox.send_keys(Keys.RETURN)
        time.sleep(2.5)
        # checkpoint
        wait = WebDriverWait(driver, 2)
        element = wait.until(EC.element_to_be_clickable((By.XPATH,'/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[4]/div[5]/div/button')))

        zoom_in = driver.find_element_by_xpath('//*[@id="widget-zoom-in"]')
        # /html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[4]/div[1]/div[3]/div/a
        # /html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div/a
        for i in range(5):
            zoom_in.click()
        results = COOR_PT.findall(driver.current_url)
        if len(results) > 0:
            latitude, longitude = results[0]
    except Exception as e:
        # print(e)
        # print(search_text)
        pass
    
    return latitude, longitude

In [35]:
miss = []
sel = 10
offset = 0.0001368

# for i in range(len(pd_list[1:2])):
df = pd_list[sel]
for j in tqdm(range(len(df[:]))):
    row = df.iloc[j]
    search_text = row.at["Name"]
    latitude = str(row.at["Latitude"])
    longitude = str(row.at["Longitude"])
    if len(latitude) < 1:
        latitude, longitude = get_coord(search_text)
        if len(latitude) > 0:
            df.at[j, "Latitude"] = latitude
            df.at[j, "Longitude"] = str(float(longitude) + offset)
        else:
            miss.append((i, j, row.at["Name"]))
    else:
        continue

# pd_list[i] = df

print(len(miss))
df.to_csv(
    "Located CSV/{0}.csv".format(files[sel]),
    quoting=csv.QUOTE_NONNUMERIC,
    escapechar="\\",
    doublequote=False,
    index=False)

100%|██████████| 672/672 [23:37<00:00,  2.11s/it]

40


In [24]:
df.head(n=3)

,index,Name,Clinic,Hospital,ST,VAC,Latitude,Longitude,Address,Contact,State
0,0,Gleneagles Medini,0,1,1,0,1.4259169,103.63692540000001,"No. 2, Jalan Medini Utara 4, Medini Iskandar, ...",,Johor
1,1,Kpjbandarmaharani Specialist Hospital,0,1,1,0,2.035902,102.56726,"No. 73-1, Jalan Stadium, 84000 Muar, Johor",,Johor
2,2,Kpjbandardato’Onn Specialist Hospital,0,1,1,0,1.5440191,103.74016820000001,"Lot Ptd 163189, Jalan Bukit Mutiara, Taman Buk...",,Johor


In [10]:
pd_list[0].to_csv(
    "Located CSV/{0}.csv".format(files[0]),
    quoting=csv.QUOTE_NONNUMERIC,
    escapechar="\\",
    doublequote=False,
    index=False)

In [82]:
COOR_PT = re.compile(r"@([\d]+[.][\d]+),([\d]+[.][\d]+),")

coords = []

for i in range(len(search_texts[:])):
    try:
        searchbox = driver.find_element_by_xpath('//*[@id="searchboxinput"]')
        searchbox.clear()
        searchbox.send_keys(search_texts[i])
        searchbox.send_keys(Keys.RETURN)
        time.sleep(3)
        try:
            wait = WebDriverWait(driver, 5)
            element = wait.until(EC.element_to_be_clickable((By.XPATH,'/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[4]/div[5]/div/button')))
        except Exception as e:
            cards = driver.find_elements_by_xpath('/html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[4]/div[1]/div[3]/div/a')
            if len(cards) > 0:
                cards[0].click()
                time.sleep(1)
            else:
                raise e

        zoom_in = driver.find_element_by_xpath('//*[@id="widget-zoom-in"]')
        # /html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[4]/div[1]/div[3]/div/a
        # /html/body/jsl/div[3]/div[10]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div/a
        for i in range(5):
            zoom_in.click()
        results = COOR_PT.findall(driver.current_url)
        if len(results) > 0:
            coords.append(results[0])
    except Exception as e:
        print(e)
        print(f"Stop: {i}")
        print(search_texts[i])
        break

Message: 

Stop: 32
Hospital Jelebu


## Others

In [4]:
driver.close()

In [38]:
options = webdriver.FirefoxOptions()
options.add_argument('user-agent={0}'.format('MQQBrowser/26 Mozilla/5.0 (Linux; U; Android 2.3.7; zh-cn; MB200 Build/GRJ22; CyanogenMod-7) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1'))
# options.add_argument('--no-proxy-server')
# options.add_argument("--proxy-server='direct://'")
# options.add_argument("--proxy-bypass-list=*")
options.headless = True
# options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
driver = webdriver.Firefox(executable_path=r'D:\Program Files\Geckodriver\geckodriver.exe', options=options)
driver.get("http://www.python.org")